In [ ]:
!pip install transformers accelerate torch PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 95.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen1.5-1.8B-Chat"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

In [ ]:
import fitz  # PyMuPDF
from google.colab import files

uploaded_file = files.upload()  # Upload PDF from your local system

pdf_path = list(uploaded_file.keys())[0]

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_text = extract_text_from_pdf(pdf_path)
print("Sample Text:\n", pdf_text[:1000])


Saving 2401.18059v1.pdf to 2401.18059v1.pdf
Sample Text:
 Published as a conference paper at ICLR 2024
RAPTOR: RECURSIVE ABSTRACTIVE PROCESSING
FOR TREE-ORGANIZED RETRIEVAL
Parth Sarthi, Salman Abdullah, Aditi Tuli, Shubh Khanna, Anna Goldie, Christopher D. Manning
Stanford University
psarthi@cs.stanford.edu
ABSTRACT
Retrieval-augmented language models can better adapt to changes in world state
and incorporate long-tail knowledge. However, most existing methods retrieve
only short contiguous chunks from a retrieval corpus, limiting holistic under-
standing of the overall document context. We introduce the novel approach of
recursively embedding, clustering, and summarizing chunks of text, constructing
a tree with differing levels of summarization from the bottom up. At inference
time, our RAPTOR model retrieves from this tree, integrating information across
lengthy documents at different levels of abstraction. Controlled experiments show
that retrieval with recursive summaries offers s

In [ ]:
def summarize_text_with_qwen(text):
    prompt = f"<|user|>\nPlease summarize the following research paper in a short and clear format:\n\n{text[:2000]}\n\n<|assistant|>"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=512, do_sample=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

summary = summarize_text_with_qwen(pdf_text)
print("Summary:\n", summary)


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Summary:
 <|user|>
Please summarize the following research paper in a short and clear format:

Published as a conference paper at ICLR 2024
RAPTOR: RECURSIVE ABSTRACTIVE PROCESSING
FOR TREE-ORGANIZED RETRIEVAL
Parth Sarthi, Salman Abdullah, Aditi Tuli, Shubh Khanna, Anna Goldie, Christopher D. Manning
Stanford University
psarthi@cs.stanford.edu
ABSTRACT
Retrieval-augmented language models can better adapt to changes in world state
and incorporate long-tail knowledge. However, most existing methods retrieve
only short contiguous chunks from a retrieval corpus, limiting holistic under-
standing of the overall document context. We introduce the novel approach of
recursively embedding, clustering, and summarizing chunks of text, constructing
a tree with differing levels of summarization from the bottom up. At inference
time, our RAPTOR model retrieves from this tree, integrating information across
lengthy documents at different levels of abstraction. Controlled experiments show
that retrie

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

embedder = SentenceTransformer("all-MiniLM-L6-v2")

def chunk_text(text, max_length=500):
    sentences = text.split('. ')
    chunks, chunk = [], ""
    for s in sentences:
        if len(chunk) + len(s) < max_length:
            chunk += s + ". "
        else:
            chunks.append(chunk.strip())
            chunk = s + ". "
    if chunk: chunks.append(chunk.strip())
    return chunks

chunks = chunk_text(pdf_text)
chunk_embeddings = embedder.encode(chunks)

# Create FAISS index
dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(chunk_embeddings))


In [ ]:
def retrieve_context(query, top_k=3):
    query_embedding = embedder.encode([query])
    _, indices = index.search(np.array(query_embedding), top_k)
    return "\n".join([chunks[i] for i in indices[0]])

def generate_rag_response(query):
    context = retrieve_context(query)
    prompt = f"""<|user|>
Based on the following content, answer the query:

Context:
{context}

Query:
{query}

<|assistant|>"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=512, do_sample=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Try asking something:
response = generate_rag_response("Summarize the main contributions of this research.")
print(response)


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


<|user|>
Based on the following content, answer the query:

Context:
URL https:
//aclanthology.org/P17-1171.
Aakanksha Chowdhery, Sharan Narang, Jacob Devlin, Maarten Bosma, Gaurav Mishra, Adam
Roberts, Paul Barham, Hyung Won Chung, Charles Sutton, Sebastian Gehrmann, et al. PaLM:
Scaling Language Modeling with Pathways.
arXiv preprint arXiv:2204.02311, 2022.
URL
https://arxiv.org/abs/2204.02311.
Arman Cohan and Nazli Goharian. Contextualizing citations for scientific summarization using
word embeddings and domain knowledge.
(Cohan & Goharian, 2017; Newman et al., 2023; Zhang et al., 2023).
Recursive summarization as Context
Summarization techniques provide a condensed view of
documents, enabling more focused engagement with the content (Angelidis & Lapata, 2018). The
summarization/snippet model by Gao et al. (2023) uses summarizations and snippets of passages,
which improves correctness on most datasets but can sometimes be a lossy means of compression.
The recursive-abstractive summa

In [ ]:
def retrieve_context(query, top_k=3):
    query_embedding = embedder.encode([query])
    _, indices = index.search(np.array(query_embedding), top_k)
    return "\n".join([chunks[i] for i in indices[0]])

def generate_rag_response(query):
    context = retrieve_context(query)
    prompt = f"""<|user|>
Based on the following content, answer the query:

Context:
{context}

Query:
{query}

<|assistant|>"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=512, do_sample=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Try asking something:
response = generate_rag_response("Easy understand about the paper")
print(response)

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


<|user|>
Based on the following content, answer the query:

Context:
An excerpt
from the story is present below and the full PDF of this story is linked here. For questions like “What
is the central theme of the story?”, an upper-level node is retrieved which includes the sentence:
“This story is about the power of human connection...
The document lengths varied significantly across the three
15
Published as a conference paper at ICLR 2024
datasets examined: QuALITY, QASPER, and NarrativeQA. Figure 5 illustrates a clear linear corre-
lation between the initial document length and the total token expenditure, emphasizing that RAP-
TOR maintains a linear token scaling regardless of document complexity or length.
Figure 6: Build time as a function of document length for documents of up to 80,000 tokens.
39–48, 2020.
URL
https://arxiv.org/abs/2004.12832.
Tom´aˇs Koˇcisk`y, Jonathan Schwarz, Phil Blunsom, Chris Dyer, Karl Moritz Hermann, G´abor Melis,
and Edward Grefenstette. The NarrativeQ